In [1]:
# Extracting all gender neutral sentences from [huggingface.co/datasets/md_gender_bias]
!pip install datasets
from datasets import load_dataset
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.8 MB/s eta 0:00:00


In [9]:
# constructing a even smaller subset with just 3k sentences
dataset = load_dataset("md_gender_bias", "convai2_inferred")
filtered = dataset.filter(lambda example: example['ternary_label']==2)
all_text = filtered["test"]['text']
print(len(filtered["test"]['text']))

df = pd.DataFrame({'text': all_text})
df.to_csv('3k_gender_neutral_data.csv', index=False)

3251


In [2]:
def remove_start_end(example):
     example['text'] = example['text'].split('> ')[1]
     example['text'] = example['text'].split(' <')[0]
     return example

In [ ]:
# function for extracting all gender neutral dataset
subset = ["convai2_inferred", "funpedia", "image_chat",
        "light_inferred", "opensubtitles_inferred", "wizard"]

all_text = []
for subdata in subset:
  dataset = load_dataset("md_gender_bias", subdata)
  if subdata in ["convai2_inferred", "light_inferred", "opensubtitles_inferred"]:
    filtered = dataset.filter(lambda example: example['ternary_label']==2)
  if subdata in ["funpedia", "wizard"]:
    filtered = dataset.filter(lambda example: example['gender']==0)
  if subdata=="image_chat":
    filtered = dataset.filter(lambda example: example['male']==example['female']) # both true & false
    '''ie. false: no gender bias in sentence
    true: sentence contains both male & female genders'''
    filtered = filtered.rename_column("caption","text") # renaming for uniformity
    filtered = filtered.map(remove_start_end) # preprocessing text column

  for split, data in filtered.items():
    all_text.extend(filtered[split]['text'])
    # data.to_csv(f"{subdata}-{split}.csv", index = None)
    '''uncomment to save the various subdata to individual csv files'''

df = pd.DataFrame({'text': all_text})
df.to_csv('all_gender_neutral_data.csv')